# Practical - 4


Name: Rohan Pillai<br>
Roll Number: 15BIT049<br>
Batch: A3<br>
Course Code: IT702<br>
Course Name: Information Retrieval Systems

## Vocabulary Creation

In [90]:
import nltk
import string

import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

#doc = ['My name is Rohan Pillai. His name is XYZ','It is raining today','I have a lot of homework to do','Today we have to complete Practical 4']
#query = 'is it raining today'
doc = ['new york times','new york post','los angeles times']
query = 'new new times'
stop_words = set(stopwords.words('english'))
stemmed_words = []

for i in doc:
    i=i.lower();
    #Removes Punctuations
    translation_table = str.maketrans("","",string.punctuation)
    #Remove \n for new line to space
    i.replace('\n', ' ')
    i = i.translate(translation_table)

    #Tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(i)
    cloud_input = ''

    #Stopwords Removal
    without_stopwords = []
    for w in tokens:
        if w not in stop_words:
            without_stopwords.append(w)
    ps = PorterStemmer()

    #Stemming    
    for w in without_stopwords:
        if ps.stem(w) not in stemmed_words:
            stemmed_words.append(ps.stem(w))
            
        
#stemmed_words.sort()
print("Vocabulary:" , stemmed_words)

Vocabulary: ['new', 'york', 'time', 'post', 'lo', 'angel']


## TF Representation

In [91]:
import string

def tf(term, document):
    return freq(term, document)

def freq(term, document):
    return document.split().count(term)

def preprocessing(i):
    stemmed_words = ''
    i=i.lower()
    #Removes Punctuations
    translation_table = str.maketrans("","",string.punctuation)
    #Remove \n for new line to space
    i.replace('\n', ' ')
    i = i.translate(translation_table)

    #Tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(i)
    cloud_input = ''

    #Stopwords Removal
    without_stopwords = []
    for w in tokens:
        if w not in stop_words:
            without_stopwords.append(w)
    ps = PorterStemmer()

    #Stemming    
    for w in without_stopwords:
        stemmed_words+=(ps.stem(w))
        stemmed_words+=" "

    return stemmed_words

doc_term_matrix = []

print("Vocabulary: ", list(stemmed_words))
doc.append(query)
print("TF Vector:")
for doc1 in doc:
    a=preprocessing(doc1.lower())
    tf_vector = [tf(word,a) for word in stemmed_words]
    tf_vector_string = ','.join(format(freq,'d') for freq in tf_vector)
    tf_vector_arr = list(tf_vector_string.split(','))
    for i in range(0,len(tf_vector_arr)):
        tf_vector_arr[i]=int(tf_vector_arr[i])
    print(tf_vector_arr)
    doc_term_matrix.append(tf_vector_arr)


Vocabulary:  ['new', 'york', 'time', 'post', 'lo', 'angel']
TF Vector:
[1, 1, 1, 0, 0, 0]
[1, 1, 0, 1, 0, 0]
[0, 0, 1, 0, 1, 1]
[2, 0, 1, 0, 0, 0]


## TF-IDF Representation

In [92]:
count_arr=[]
doc_term_matrix1 = []
for i in range(0, len(doc_term_matrix[0])):
    count=0
    for j in range(0, len(doc_term_matrix)-1):
         if(doc_term_matrix[j][i]>0):
                count+=1
    count_arr.append(count)
print("Count Array: ", count_arr)

def idf(word, doc):
    noOfSamples = len(doc)
    df = count_arr[stemmed_words.index(word)]
    return np.log(noOfSamples/1+df)

my_idf_vector = [idf(word,doc) for word in stemmed_words]
print("IDF Vector: ", my_idf_vector)

for i in range(0,len(doc_term_matrix)):
    doc_term_matrix1.append([a*b for a,b in zip(doc_term_matrix[i],my_idf_vector)])
    
print("")
print("TF-IDF Vector: \n", np.matrix(doc_term_matrix1))

query = doc_term_matrix1[-1]
print("\nQuery Vector: ", query)

Count Array:  [2, 2, 2, 1, 1, 1]
IDF Vector:  [1.791759469228055, 1.791759469228055, 1.791759469228055, 1.6094379124341003, 1.6094379124341003, 1.6094379124341003]

TF-IDF Vector: 
 [[1.79175947 1.79175947 1.79175947 0.         0.         0.        ]
 [1.79175947 1.79175947 0.         1.60943791 0.         0.        ]
 [0.         0.         1.79175947 0.         1.60943791 1.60943791]
 [3.58351894 0.         1.79175947 0.         0.         0.        ]]

Query Vector:  [3.58351893845611, 0.0, 1.791759469228055, 0.0, 0.0, 0.0]


## Cosine Calculation

In [93]:
import math

print(query)

def lengthOfVec(list):
    sum = 0
    for i in list:
        sum+=i**2
    return math.sqrt(sum)

print("\nLength:")
for i in doc_term_matrix1:
    indexDoc = doc_term_matrix1.index(i)
    if indexDoc==len(doc_term_matrix1)-1:
        print("Query: ", lengthOfVec(i),"\n")
    else:
        print("Doc ", indexDoc+1, ": ", lengthOfVec(i))
        

final_result = []
indexDoc = 0
for i in doc_term_matrix1:    
    if indexDoc==len(doc_term_matrix1)-1:
        break
    else:
        a1 = [a*b for a,b in zip(i,query)]
        final_result.append(sum(a1)/lengthOfVec(i))
    indexDoc+=1

print("Cosine Similarity between Document and Query: ",final_result)
print("\nMost similar document is Document ",final_result.index(max(final_result))+1)

[3.58351893845611, 0.0, 1.791759469228055, 0.0, 0.0, 0.0]

Length:
Doc  1 :  3.1034184356456356
Doc  2 :  3.0018484946973985
Doc  3 :  2.896719313901309
Query:  4.006495972522873 

Cosine Similarity between Document and Query:  [3.1034184356456356, 2.1389500511031128, 1.1082889461059393]

Most similar document is Document  1
